# Predicting without converting the MNIST training images to files on disk

As convenient as it may see, leveraging also code I have used before, converting the `csv`-file to `png`-files on disk feel unneccessary.

It took a while to make it work, but finally here we go: In this approach, I make the predictions directly on the `csv`-file.

In [2]:
from fastai.vision.all import *

## PIL Image vs. Fast.AI image

A standard PIL image (PIL = Python Imaging Library) has the type `PIL.Image.Image`. This type, can, however, not be used im combination with `learn.predict`. It expects other types, for example, the Fast.AI interpretation of an image `fastai.vision.core.PILImage`. Therefore, the data needs to be converted into the right format before being able to make a prediction.

### Failure mode: Why predict is not possible

Let's demonstrate the failure so that we know what we are taking about

In [3]:
import pandas as pd

test = pd.read_csv("test.csv")
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def get_test_image(index):
    im = torch.tensor(test.iloc[index])
    image = Image.frombytes('L', (28, 28), bytes(im))
    return image

In [5]:
image = get_test_image(0)
image

In [6]:
type(image)

PIL.Image.Image

Here we go, we have an image of type `PIL.Image.Image`, let's try to call `learn.predict` on it.

In [10]:
learn = load_learner('../mnist1-fastai/export.pkl')
#learn.predict(image)

If you uncommented and executed the above line, the result would be an error and the following `AssertionError` would appear:
    
```
AssertionError: Expected an input of type in 
  - <class 'pathlib.PosixPath'>
  - <class 'pathlib.Path'>
  - <class 'str'>
  - <class 'torch.Tensor'>
  - <class 'numpy.ndarray'>
  - <class 'bytes'>
  - <class 'fastai.vision.core.PILImage'>
 but got <class 'PIL.Image.Image'>
 ```

While other options would be possible, I tried to create an image of type `fastai.vision.core.PILImage`.

## Failure mode: Converting `PIL.Image.Image`to `fastai.vision.core.PILImage`

This seemingly easy task was a challenge to me. Even the forums did not help (because I did not read to the end): https://forums.fast.ai/t/how-to-convert-a-pil-image-to-a-fastai-image-object/68407/8

Finally, I found a way in the docs: https://docs.fast.ai/vision.core.html#pilimagebw.

In [13]:
# Step 1: create a tensor from the image:
timg = TensorImage(image2tensor(image))
#timg

# Step 2: Create a new image from the tensor
#tpil = PILImage.create(timg)
#print(type(tpil))
#tpil

But still this does not work, because we get a TypeError:

`TypeError: Cannot handle this data type: (1, 1, 1), |u1`

The reason is that the image is stored in the wrong format. Mode `L` (as explained [here](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)) just creates a grey-scale image and the methods expect an RBG image.

### Solution: Converting `PIL.Image.Image`to `fastai.vision.core.PILImage`

Let's solve this by converting the image:

In [14]:
image = image.convert('RGB') # this is the magic

Now the above code runs and converts the image:

In [16]:
# Step 1: create a tensor from the image:
timg = TensorImage(image2tensor(image))

# Step 2: Create a new image from the tensor
tpil = PILImage.create(timg)
print(type(tpil))
tpil

<class 'fastai.vision.core.PILImage'>


### Solution: function `pil2fast` / `pil2fast2`

Let's wrap this in a function to close the topic for now

In [24]:
def pil2fast(pil):
    timg = TensorImage(image2tensor(pil))
    fast = tpil = PILImage.create(timg)
    return fast

In [27]:
pil = get_test_image(0).convert('RGB')
print(type(pil))
pil

<class 'PIL.Image.Image'>


In [28]:
fast = pil2fast(pil)
print(type(fast))
fast

<class 'fastai.vision.core.PILImage'>


In [56]:
def pil2fast2(pil):
    return PILImage.create(np.array(pil.convert('RGB')))

In [57]:
fast2 = pil2fast2(pil)
print(type(fast2))
fast2

<class 'fastai.vision.core.PILImage'>


`pil2fast2` is the faster version.

## Back to Kaggle

With all the conversion out of the way, let's apply that to the Kaggle competition.

In [58]:
def get_fast_test_image(index):
    im = torch.tensor(test.iloc[index])
    pil = Image.frombytes('L', (28, 28), bytes(im))#.convert('RGB')
    fast = pil2fast2(pil)
    return fast

In [59]:
test_image = get_fast_test_image(0)
test_image

In [60]:
type(test_image)

fastai.vision.core.PILImage

In [61]:
def classify_image(img):
    pred,pred_idx,probs = learn.predict(img)
    return dict(zip(learn.dls.vocab, map(float, probs)))

In [62]:
classify_image(test_image)

{'0': 6.806673127357499e-07,
 '1': 9.510736731499492e-08,
 '2': 0.9999420642852783,
 '3': 3.5234781535109505e-05,
 '4': 1.0088542268249512e-07,
 '5': 3.694026190714794e-06,
 '6': 8.713337962262813e-08,
 '7': 9.428195880900603e-06,
 '8': 4.397437351144617e-06,
 '9': 4.274736966181081e-06}

In [63]:
def get_prediction(img):
    pred, _, _ = learn.predict(img)
    return pred

In [64]:
get_prediction(test_image)

'2'

In [69]:
def create_submission():
    submission = pd.DataFrame(columns=['ImageId', 'Label'])

    image_ids = []
    preds = []


    for i in range(len(test)):
#    for i in range(20):
        test_image = get_fast_test_image(i)
        pred = get_prediction(test_image)

        image_ids.append(i+1)
        preds.append(str(pred))   

    submission['ImageId'] = image_ids
    submission['Label'] = preds
    return submission

In [67]:
#submission = create_submission()

In [68]:
#submission.to_csv('submission.csv', index=False)

Not surprising, the result was the same as with the first submission:

![My second submission result](sub2-099442.png)